In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import time
import copy
import string
import json
import pickle as pk
import math

from treelib import Node, Tree

from collections import Counter

from math import sin, asin, cos, radians, fabs, sqrt

In [2]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a+')

    def write(self, message):
        self.terminal.write(message)
        self.terminal.flush()
        self.log.write(message)
        self.log.flush()

    def flush(self):
        pass

#logfile_3 =[".\\log\\stay_point_exact_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
#sys.stdout = Logger(logfile_3[0], sys.stdout)
#filename2 =[".\\log\\a_stderr_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log_file']
    
#sys.stdout = Logger(".\\a_stdout.log", sys.stdout)
#sys.stderr = Logger(".\\a_stderr.log_file", sys.stderr)             # redirect std err, if necessary

#sys.stderr = Logger(filename2[0], sys.stderr)   

In [3]:
############################    从文件中 读取 数据   ##############
def read_q(filename):
    f = open(filename,"rb+")
    st = pk.load(f)
    f.close()
    return st

In [4]:
##########################    向文件中  写 数据  #####################
def write_q(st, filename):
    f = open(filename, "wb+")
    pk.dump(st,f)
    f.close()

In [5]:
#######################   Dis()  根据经纬度计算距离 distance(a)  ####################3
#     input:  a  2*2 两个点的经纬度坐标；
#     output:  距离  m

################    测试用例   #################
#a = [[40.009532,116.322153],[40.009493,116.322103]]
#b = pd.DataFrame(a)
#Dis(b)

EARTH_RADIUS = 6371.004           #地球平均半径，6371km
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def Dis(b): 
#def get_distance_hav(lat0, lng0, lat1, lng1):
    """用haversine公式计算球面两点间的距离。"""
    # 经纬度转换成弧度
    a = pd.DataFrame(b)
    lat0 = radians(a.iloc[0,0])
    lat1 = radians(a.iloc[1,0])
    lng0 = radians(a.iloc[0,1])
    lng1 = radians(a.iloc[1,1])
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    dist = 2 * EARTH_RADIUS * asin(sqrt(h))*1000
 
    return dist

In [6]:
def tra_cos(a):
    b = a[0]
    c = a[1]
    d = a[2]
    
    bc = Dis([b,c])
    bd = Dis([b,d])
    cd = Dis([c,d])
    
    t1t2 = math.pow(bc, 2)
    t1r2 = math.pow(bd, 2)
    t2r2 = math.pow(cd, 2)
        
    cosva = (t1t2 + t1r2 - t2r2)/ (2*bc*bd)
    
    return cosva

In [7]:
###################################   单停留点与 POI的匹配    ############
# 输入:  停留点list (一个 三元组) , st_poi_dic 字典list  [{},{},{}]
# 输出： 最近的POI  及 停留时间  和 语义类型    五元组[id, lat,lng,time,category]
#st_p = st_p_list[0]

def match_1(st_p, sp_dic):
    poi_st_1 = [] 
    for i in range(0, len(sp_dic)):
        a = [[sp_dic[i]["location"]["lat"], sp_dic[i]["location"]["lng"]],[st_p[0],st_p[1]]]
        b = pd.DataFrame(a)
        c = Dis(b)
        poi_st_1.append(c)    
    #print(len(poi_st_1)) 
    j = poi_st_1.index( min(poi_st_1) )
    return  [sp_dic[j]["id"],sp_dic[j]["location"]["lat"], sp_dic[j]["location"]["lng"], st_p[2],sp_dic[j]["category"]]
    

In [8]:
###################  match()  停留点与语义树叶子结点的匹配  ##############################
# 输入 si_h_f[i]   一维list ; st_l   语义树的所有叶子结点
# 输出 sit_l    匹配后的叶子结点

#si = ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1]
#st_l = st.leaves()
#n_id = match(si, st_l)
#st.get_node(n_id)

def match_2(si, st_l):
    for i in range(len(st_l)):
    
        if st_l[i].data == None:
            continue
        else:
        
            if si[0]==st_l[i].data["id"] and si[1]==st_l[i].data["location"]["lat"] and \
            si[2]==st_l[i].data["location"]["lng"]:
                n_id = st_l[i].identifier
                break

            
    return n_id       

In [9]:
from math import radians, cos, sin, asin, sqrt
def distance_vec(lon1, lat1, lon2_v, lat2_v): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)   """
    # 将十进制度数转化为弧度
    lon1, lat1 = map(radians, [lon1, lat1])
    lon2_v=np.radians(lon2_v)
    lat2_v=np.radians(lat2_v)
    dlon = lon2_v - lon1 
    dlat = lat2_v - lat1 
    a = np.power(np.sin(dlat/2),2) + cos(lat1) *np.cos(lat2_v) *  np.power(np.sin(dlon/2),2)#**2
    c = 2 *np.arcsin(np.sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    dis_l=c * r * 1000
    #print(dis_l.argmin())
    return dis_l

In [10]:
##########################    生成K-匿名集  ##################
## 输入 当前被保护 POI si;   候选匿名集 m_l
## 输出 m_2     计算出与被保护位置之间的距离并排序后的候选集   [{},{},{},{}] 在node.data["distance"],添加了距离字段

#si = ['北京郊野园中园', 39.996209, 116.26688600000001, 13.75000013387762, 'farm', 1]


def Cal_dis_sort(si, m_l):
    
    k_as = []
    #dis = []
    m_l2 = copy.deepcopy(m_l)
    
    #print("K-匿名集生成前叶子结点list长度： ",len(m_l2))

    ####  距离函数重写    #######
    lat = si[1]
    lng = si[2]
    
    m_l2_loc = []
    for i in range(len(m_l2)):
        n_d = m_l2[i].data
        a = []
        a = [n_d["location"]["lat"],n_d["location"]["lng"]]
        m_l2_loc.append(a)
        
    df_ml_list=pd.DataFrame(m_l2_loc,columns=['la','lo'])
    la_l=np.array(df_ml_list['la'])
    lo_l=np.array(df_ml_list['lo'])
    
    dis_l = distance_vec(lng, lat, lo_l, la_l)
    #print(dis_l)
    
    for l in range(len(m_l2)):    # 添加距离字段
        m_l2[l].data["distance"] = dis_l[l]
        
    ##########
            
    #print("计算距离后，打印第一个叶子结点： ", m_l2[0])
    m = sorted(m_l2,key = distance)      #自定义函数按列表f中字典的distance从小到大排序， 返回结点 list    
    del(m[0])
    
#    if len(m) < k:
#        k_as = m
#    else:
#        for j in range(k):
#            k_as.append(m[j])
            
    return m  

def distance(s): 
    return s.data['distance'] 

In [11]:
####################   proba()  把 staying list  转换为概率特征    ############
#  输入: staying list   一个POI上的停留时间list ;  把停留时间化为了9个时段0::8
#  输出: st_f      停留时间概率分布，  
def proba(st):
    t = st.copy()
    for i in range(0, len(st)):
    
        if 10 <= st[i] < 30:
            t[i] = 0
        elif 30 <= st[i] < 60:
            t[i] = 1
        elif 60 <= st[i] < 90:
            t[i] = 2
        elif 90 <= st[i] < 120:
            t[i] = 3
        elif 120 <= st[i] < 150:
            t[i] = 4
        elif 150 <= st[i] < 180:
            t[i] = 5
        elif 180 <= st[i] < 210:
            t[i] = 6
        elif 210 <= st[i] < 240:
            t[i] = 7
        else:
            t[i] = 8
    
    b = len(t)
    a = Counter(t)
    #print(a)
    stf = [0]*9        # staying list 转化为 fecture vector
    #print(st_f)

    for key, value in a.items():
        stf[key] = round(value/b, 3)
        
    return stf   

In [12]:
def tra_cos(a):
    #b = a[0]
    #c = a[1]
    #d = a[2]
    
    bc = a[0]
    bd = a[1]
    cd = a[2]
    
    t1t2 = math.pow(bc, 2)
    t1r2 = math.pow(bd, 2)
    t2r2 = math.pow(cd, 2)
        
    cosva = (t1t2 + t1r2 - t2r2)/ (2*bc*bd)
    
    return cosva

In [13]:
##################################     读文件   换用户     ##########################################################
##########  读个人 st_poi_pro_p    语义重要性   ##########     用户： 000
i = 0 
filename = ".\\os_file\\public_data\\"+str(i)+"\\st_poi_pro_p"+str(i)+".pkl"
st_poi_pro_p = read_q(filename)
print(len(st_poi_pro_p))

5


In [14]:
###############  读文件，准备数据  ###########################
##########  总停留点  #####################
filename = ".\\os_file\\public_data\\st_pa.pkl"       #三重列表 [【user】【file】【st_p】]
st_pa = read_q(filename)
print(len(st_pa))

########## 总 st_poi_pro  语义重要性  ###############
filename = ".\\os_file\\public_data\\st_poi_pro.pkl"   #{'enter':{'bar':0.0041,'bath':0.03082}{}}{}{}{}{}
st_poi_pro = read_q(filename)
print(len(st_poi_pro))

##########  st_poi_dic  总ST-POI dict-list字典列表 ##################
filename = ".\\os_file\\public_data\\st_poi_dic.pkl"
st_poi_dic = read_q(filename)
print(len(st_poi_dic))

182
5
3634


In [16]:
##############################   主程序   4*30（120） ----比较实验 ####################
## 输入 st_pa   所有停留点  三层list  [user][file][st_p]
## 输出 st_poi_f  五元组list   [id, lat, lng,time, category]

#### 从总停留点 st_pa 提取用户文件的停留点
##   用户： 000；  文件： 48， 80， 138， 147， 168 (j = 168-1)    
##   
"""[	"s_cluster"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_2"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_2"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_2"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_2"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_2"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_2"	,	"r_6"	,	3	,	6	],
[	"s_cluster"	,	"0_3"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_3"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_3"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_3"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_3"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_3"	,	"r_6"	,	3	,	6	]"""


big_t = [  
    
[	"s_cluster"	,	"0_1"	,	"k_2"	,	2	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_4"	,	4	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_5"	,	5	,	3	],
[	"s_cluster"	,	"0_1"	,	"k_6"	,	6	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_2"	,	3	,	2	],
[	"s_cluster"	,	"0_1"	,	"r_3"	,	3	,	3	],
[	"s_cluster"	,	"0_1"	,	"r_4"	,	3	,	4	],
[	"s_cluster"	,	"0_1"	,	"r_5"	,	3	,	5	],
[	"s_cluster"	,	"0_1"	,	"r_6"	,	3	,	6	]

]



"""kd = 2        # k 匿名度
o_r = 3     #  最高高度
o_l = 1     #  最低高度
cluster = "mb_kmeans"    # 聚类器
ex_times = "0_1"      #第几次实验
at = "k_2"      # 参数设置
i = 0
"""


logfile = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\logfile.txt"

sys.stdout = Logger(logfile, sys.stdout)

i = 0
o_l = 1     #  最低高度
num = 0
num_tot = 0

dis_res = 1500    # 距离阈值
#dir_res = 90      # 方向阈值


for bb in big_t:  #参数设置
    cluster = bb[0]   # 聚类器  a_cluster,,mb_kmeans
    ex_times = bb[1]      #第几次实验
    at = bb[2]
    kd = bb[3]        # k 匿名度
    o_r = bb[4]    #  最高高度
    print(cluster, ex_times, at,kd,o_r)
    
    ########## semantic_tree    ###############
    filename = ".\\os_file\\st_poi_clusters\\"+str(cluster)+"\\semantic_tree_6.pkl"
    st = read_q(filename)

    wz = [47,79,137,146,167]
    for aa in wz:      # 选择文件， 选择目录
        j = aa
        #j = 167   # 第48个文件  48   138   168

        filename1 = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\st_poi_f_"+str(i)+"_"+str(j)+".pkl"
        filename2 = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_l_f_"+str(i)+"_"+str(j)+".pkl"
        filename3 = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\si_h_f_"+str(i)+"_"+str(j)+".pkl"
        filename4 = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\k_as_"+str(i)+"_"+str(j)+".pkl" 
        filename5 = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_tra_"+str(i)+"_"+str(j)+".pkl"
        filename6 = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\rec_cos_"+str(i)+"_"+str(j)+".pkl"
        filename7 = ".\\os_file\\compa_ex\\privacy_protection\\res_6\\"+str(cluster)+"\\"+str(ex_times)+"\\"+str(at)+ "\\tra_leaf_"+str(i)+"_"+str(j)+".pkl"

        #############################################################
        #  输入  st_pa 所有停留点   三重列表
        #  输出  st_pa_f  文件的停留点    一维列表  取文件中的停留点

        st_pa_f = []
        #st_p = []    #  停留点的list 

        m = st_pa

        if len(m[i]) == 0:            #  如果  user i 没有文件 
            print("该用户没有轨迹文件")
            sys.exit()
        else:       
            if len(m[i][j]) == 0:       #  文件内没有stay point，
                print("该文件内没有停留点")
                sys.exit()
            else:      #  文件内有 stay point               #  user_file_st-p
                for k in range(0, len(m[i][j])):
                            # 对用户 i 文件 j, 停留点 k, st_p[i][j][k]     停留点
                    a = []
                    a = m[i][j][k]
                    st_pa_f.append(a)
            
        b = 0
        b = len(st_pa_f)
        num_tot += b
        print("用户： ",i," 在文件： ",j," 中共有停留点有： ", b,"  个")

        #################  匹配 st_pa_f ----> st_poi_f ############### 
        ############   把 文件停留点， 经 st_poi_dic 匹配, 得 POI+ category： st_poi_f
        # 输入  文件停留点st_pa_f,    st_poi_dic   停留点与POI匹配的字典列表
        # 输出  st_poi_f      文件与POI匹配列表   二维列表
        # [['无尽', 40.005973, 116.32533400000001, 65.81666663987562, 'scenic'],

        st_poi_f = []
        for l in range(0, len(st_pa_f)):
        #for i in range(0, 20):
    
            st_p_m = match_1(st_pa_f[l], st_poi_dic)
            st_poi_f.append(st_p_m)

        write_q(st_poi_f, filename1)
        print("已写入 st_poi_f 文件")

        ##############################  semantic importance  ##################
        ## 输入 st_poi_f   两维-五元组-list   [[id, lat, lng,time, category],  ,  ] ;    st_poi_pro_p, st_poi_pro
        ## 输出 si_l_f     两维-六元组-list   [id, lat, lng,time, category,level]
        # 输出文件 si_l_f_0_47.pkl   

        f = st_poi_f
        pro_p = st_poi_pro_p
        pro = st_poi_pro

        si_l_f = []
        for k in range(len(f)):
    
            ca = f[k][4]
    
            flag = 0
    
            for item1 in pro:
        
                if flag == 1:
                    break
                else:
            
                    for item2 in pro[item1]:
                        if flag == 1:
                            break
                        else:
                            if ca == item2:         # 判断语义类型匹配
                                flag = 1
                                a = 0
                                a = pro[item1][item2]
                                b = 0
                                b = pro_p[item1][item2]
                        
                                if a == 0:
                                    print("st_poi_pro 没有该类型的停留点")
                            
                                c=0
                                c = b * math.log(1/a)
                                                        
                                d = [f[k][0],f[k][1],f[k][2],f[k][3],f[k][4],c]
                                si_l_f.append(d)
                    
        write_q(si_l_f, filename2)
        print("已写入 si_l_f 文件")
        ###########################  si_h_f 语义树高度   #######################
        ## 输入 si_l_f， o_l, o_r   
        ## 输出 si_h_f     六元组-两维list   [[id, lat, lng,time, category,height], , ]
                  # 输出文件 si_h_f_0_47.pkl 

        al = []
        for k in range(len(si_l_f)):
            al.append(si_l_f[k][5])
        w = max(al)

        si_h_f = []
        for k in range(len(si_l_f)):
            a = si_l_f[k][5]
            b = a/w
            c = 1-b
    
            l = b * o_l + c *o_r
            h = round(l)
            print("用户",i,"文件",j,"停留序号",k,"回溯高度",h)
    
            d = []
            d = [si_l_f[k][0],si_l_f[k][1],si_l_f[k][2],si_l_f[k][3],si_l_f[k][4],h]
            si_h_f.append(d)

        write_q(si_h_f, filename3)
        print("已写入 si_h_f 文件")
        #####################  轨迹重构 rec_tra  rec_tra_i_j.pkl ################
        
        
        
        ####  构建 K-匿名集  ################
        # 输入  si_h_f 文件内停留点的语义高度； st 语义树；
        # 重构轨迹 rec_tra 

        si = si_h_f    # 文件中停留点list 6元组
        st_l = st.leaves()   # st 叶子结点list

        rec_tra = []   # 存储 重构的轨迹
        k_as = []      # 存储 K 匿名集
        rec_cos = []    #  角度
        tra_leaf = []   # 存储匹配的原始叶子结点
        for k in range(len(si)):      # 轨迹中的每一个结点      一个文件为一条轨迹
            
            k_as_can = []

            h = si[k][5]      # 语义匿名高度
            n_id = match_2(si[k], st_l)     # 文件停留点与叶子结点匹配，   返回匹配结点  id
            sp_node = st.get_node(n_id)
            tra_leaf.append(sp_node)
            #print(st.level(n_id)+1, h)
            m_id = n_id
            if st.level(m_id) < h:
                m_id = "poi"
            else:
                for l in range(h):
                    #print(l)
                    m_id = st.parent(m_id).identifier   # 返回 中间结点 id
    
                    #print(m_id)
            m_l = st.leaves(m_id)    # 返回 中间结点 对应的 叶子结点  匿名集  [node, node, node]
            ##############  删除 空结点  #############
            ai = 0
            while ai < len(m_l):     #  删除空叶子结点
        
                if m_l[ai].data == None:
                    del m_l[ai]
                else:
                    ai += 1
   
            ########################################
            for l in range(len(m_l)):         ## staying 特征概率化
                abc = m_l[l].data["staying"]
                bcd = proba(abc)
                m_l[l].data["staying"] = bcd
            ########################### 选点  ##############################   
            #  对轨迹第一个点，生成点只要在1500范围内即可
            #  对轨迹中连续的相同点，条件同上
            #  对轨迹中非同一点，角要小于 53 度
            #   kd  k-匿名的大小
            
            #k_as_m = gen_k_as(si[k], m_l)       # 生成K-匿名集，  [node,node,node]
            
            m_2 = Cal_dis_sort(si[k], m_l)      # 对叶子结点列表，计算距离并排序
            
           
            #print("匿名集的大小", len(k_as_m))
            #print("已生成 si_h_f:")
            
            t2 = [si[k][1],si[k][2]]  
            print("文件：",j,"当前是第",k,"个点")
            
            if len(m_2) == 0:         #  如果匿名集为空，返回原结点
                recon_node = sp_node
                num += 1
            else:
                

                if k == 0:               # 对轨迹第一个结点，要求在 1.5km 内生成 fake 结点
                                    
                    q = 0    
                    for aa in m_2: 
                        q = q+1
#                        r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                        dis = aa.data["distance"]
                        aa.data["cos"] = -1
                      
                        if dis < dis_res:
                            k_as_can.append(aa)
                    
                        if dis > dis_res:
                            break
                        if q >= kd:
                            break
                            
                    #k_as.append(k_as_can)       # 返回并保存该结点的K-匿名集， 两维列表 【第几个结点】【fake location】
                            
                else:           # 不是轨迹第一个节点
                    t1 = [si[k-1][1],si[k-1][2]]
                    t1_t2 = Dis([t1,t2])
            
                    if t1_t2 == 0:         #  如果与上一结点是同一个停留点，则只要距离不超过  1500 米就行
                        print("该轨迹点与前一个点相同")
                        
                        q = 0
                        # k_as_can = []
                        for aa in m_2: 
                            q = q+1
    #                        r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            dis = aa.data["distance"]
                            aa.data["cos"] = -1
                      
                            if dis < dis_res:
                                k_as_can.append(aa)
                    
                            if dis > dis_res:
                                break
                            if q >= kd:
                                break
                            
                        #k_as.append(k_as_can)       # 返回并保存该结点的K-匿名集， 两维列表 【第几个结点】【fake location】
                        
                    else:
                        print("该轨迹点既不是第一个，与前一个点也不相同")
                   
                        ti = 0
                        for aa in m_2:
                            
                            r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            t1_r2 = Dis([t1,r2])
                            t2_r2 = aa.data["distance"]
                            
                            if t1_r2 == 0:     #       如果 fake 位置与前一个位置相同，则放弃
                                m_2[ti].data["cos"] = -1
                            else:                   #   正常结点， 在90度偏移范围内选择 fake 结点 
                                dista = [t1_t2, t1_r2, t2_r2]
                                ttcos = tra_cos(dista)
                                m_2[ti].data["cos"] = ttcos
                            
                            ti = ti+1
                                
                        
                        q = 0     
                        for aa in m_2: 
                            q = q+1
    #                        r2 = [aa.data["location"]["lat"],aa.data["location"]["lng"]]
                            dis = aa.data["distance"]
                      
                            if dis < dis_res and aa.data["cos"] > 0:  
                    
                                k_as_can.append(aa)
                    
                            if dis > dis_res:
                                break
                            if q >= kd:
                                break
                            
                        #k_as.append(k_as_can)                  
                        
            if len(k_as_can) == 0:                 # 判断候选匿名集是否为空                
                recon_node = sp_node
                tcos = -1
                rec_cos.append(tcos)
                num += 1
                
            else:
                idx = np.random.randint(len(k_as_can))      # k范围内，随机生成索引
                recon_node = k_as_can[idx]
                
                tcos = k_as_can[idx].data["cos"]
                rec_cos.append(tcos)         

            rec_tra.append(recon_node)
            k_as.append(k_as_can)      # 返回并保存该结点的K-匿名集， 两维列表 【结点i】【fake node j】
            print("文件：", j,"中，第：",k,"个结点有",len(k_as_can),"个匿名结点" )
        
        write_q(tra_leaf, filename7)
        write_q(rec_tra, filename5)
        write_q(k_as,filename4)
        write_q(rec_cos,filename6)
        print("用户在文件: ",j,"中共有重构位置：",len(rec_tra),"个")
print("在为用户重构轨迹过程中共保留原始结点：",num,"个；重构结点总数为：",num_tot,"个；比例为：",round(num/num_tot,4))